(cont:mm:dynamics)=
# Paging Dynamics
## Boot-time
When a paged virtual memory system is booted one for the first actions is to insert every page of physical memory onto a free page list.  As the boot sequence continues the kernel needs to allocate memory for kernel text and data therefore some of the free pages must be allocate from the free list.  By the time the boot sequence is complete typically 10%-15% of the total memory will used by the kernel for the life the running system.
## Consuming physical memory
### Kernel page wiring
Most or all of the memory pages allocated by the kernel especially during the boot sequence are permanent in other words will never be released or freed to be used any other process, these are known as wired pages.  However, as the system runs and new processes, files and network connections are created, etc. the kernel temporarily allocates memory for data structures.  When processes exit, files are deleted and network connections are terminated the associated data structures are freed and the kernel frees the underlying memory pages.
  
### Page faults
The virtual address space of a process is typically very large and but sparsely utilized.  For example, the typical x86_64 address space is hundreds of TB in size but typically only a few GB are actually allocated and only a few MB of that are actually used.  Virtual memory is allocated in the address space by a process using either the sbrk() or mmap() system calls.  Newly allocated virtual memory has no physical memory mapped into it until the process references it and a page fault occurs.  The page fault handling routines allocate free pages of memory and map them into the virtual memory on demand.  This means that not only is the whole address space sparsely populated with valid virtual memory regions but even those are sparsely populated with physical memory.  Over time the collective page faults for every running process consume a significant amount of physical memory on the system.

### Buffer Cache
As was mentioned earlier in this chapter, the Buffer Cache uses free pages of physical memory to cache file system data and meta-data.  When a file is opened and any block of that file is read or written the file system will typically read several blocks of that file into free pages managed by the Buffer Cache, thereby eliminating most of the future synchronous IO associated with the file.  The Buffer Cache continues to cache a file's data and meta-data after the file is closed and is no longer being referenced in case the file is re-opened its data will cached in memory.

## Freeing physical memory
### Process Exiting
When a process exits all of the data structures associated with that process are freed.  This potentially frees several pages that the kernel wired for this process.  In addition all of the virtual memory regions within the processes virtual address space are free.  Since the anonymous regions are private and specific to the process all the physical memory pages mapped into these virtual regions are immediately freed.  However, the mapped file regions only contain mapped Buffer Cache pages so they are not freed until the Buffer Cache logic decides to do so.

### Page reclaiming
After a system has been up for a while several processes are created and many have exited.  While running these
processes allocate memory, invoke page faults, open, read write and close several files, etc.  After a while free memory pages become depleted and the system must reclaim active pages of memory.
 
#### Paging
#### Swapping
#### Page cache purging
- System shutdown